### A Brief History of Generative Recurrent Networks

![A Brief History of Generative Recurrent Netowrks](./history_grn.png)

### How to Generate Sequence Data

- the universal way to generate sequence data in deep learning is to train a network (usually an RNN or a convnet) to predict the next token or next few tokens in a sequence
- as usual when working with text data, tokens are typically words or characters, and any network that can model the probaility of the next token given the previous ones is called a language model; a language model captures the latent space of language: its statistical structure

once trained a language model, you can sample from it (generative new sequences): feed it an initial string of text (called conditioning data), ask it to generate the next character or the next word (you can even generate several tokens at once), add the generaed output back to the input data, and repeat the process many times 

![The Process of Character-by-Character Text Generation Using a Language Model](./process_language_model.png)

### The Importance of the Sampling Strategy

in order to control the amount of stochasticity in the sampling process, we'll introduce a parameter called the 'softmax temperature' that characterizes teh entropy of the probability distribution used for sampleing: it characterizes how surpirsing or predictable the choice of the next character will be 

given a temperature value, anew probability distribution is computed from the original one (the softmax output of the model) by reweighting it in the following way

In [ ]:
# reweighting a probability distribution to a different temperature
import numpy as np

def reweight_distribution(original_distribution, temperature=0.5): 
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

![Different Reweightings of One Probability Distribution; Low Temperature = more deterministic, High Temperature = more random](./temperature.png)

### Implementin Character-Level LSTM Text Generation

##### 中文词向量

In [1]:
import numpy as np
import os

embeddings_index = {}
f = open('C:/Users/jim_c/Desktop/Task/Chinese-Word-Vectors/sgns.financial.bigram-char', encoding='utf8')
for line in f:
    while 1:
        try:
            line = f.readline()
        except:
            pass
        if (line !=''):
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:])
            embeddings_index[word] = coefs
        else:
            break
f.close()
    
print('Found %s word vectors.' % len(embeddings_index))

Found 467149 word vectors.


In [2]:
embeddings_index['惠丰']

array(['-0.356612', '-0.279084', '-0.126416', '0.628335', '0.551051',
       '0.335221', '-0.880320', '0.223030', '0.112443', '0.001736',
       '0.120411', '-0.108307', '-0.600671', '-1.378021', '0.335731',
       '0.160973', '0.228364', '-0.403491', '-0.659103', '0.653530',
       '0.832316', '-0.670925', '1.804369', '0.279993', '-0.859297',
       '0.597914', '1.167754', '0.733485', '-0.073254', '-0.481956',
       '-0.265198', '-0.642714', '0.292259', '-0.426623', '0.389194',
       '-1.224456', '1.046738', '0.827610', '0.200318', '0.126715',
       '-0.832044', '0.542205', '0.575616', '0.069012', '-0.315015',
       '-0.285998', '-0.351620', '-0.336552', '-0.676042', '0.550532',
       '-0.146298', '-0.496723', '-0.326674', '-0.568264', '0.609188',
       '0.258278', '0.025612', '-0.440745', '-0.411169', '-0.797708',
       '0.021110', '-0.259782', '0.104928', '-0.565511', '-0.208318',
       '0.522938', '0.358395', '0.063634', '-0.514831', '0.461302',
       '-0.202765', '1.00793

##### Building the Network

single LSTM layer followed by a Dense classifier and sorfmax over all possible characters; but note taht recurrent neural networks aren't the only way to do sequence data generation; 1D convnets also have proven extremely successul at this task in recent times

In [2]:
# single-layer LSTM model for next-character prediction
import keras
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(100, len(embeddings_index))))
model.add(layers.Dense(len(embeddings_index), activation='softmax'))

optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

##### Training the Language Model and Sampling from it

given a trained model and a seed text snippet, you can generate new text by doing the following repeatedly:
- draw from the model a probability distribution for the text character, given the generated text available so far
- reweight the distribution to a certain temperature
- sample the next character at random according to the reweighted distribution
- add the new charavter at the end of the available text

In [3]:
# function to sample the next character given the model's predictions
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

the loop repeatedly trains and generates text, using a range of different temperature after evey epoch; this allows to see how the generated text evolves as the model begins to converge, as well as the impact of temperature in the sampling strategy

In [ ]:
# text-generation loop
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    generated_text = []
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('-------temperature:'. temperature)
        sys.stdout.write(generated_text)
        for i in range(400):
            sampled = np.zeros((1, 100, 467149))
            for t